In [3]:
import joblib
import MeCab
import numpy as np
import pandas as pd

In [4]:
# データの読み込み
df = pd.read_csv("wikipedia-train.txt",sep="\t")

In [5]:
df.head()

,category,text
0,mathematics,くし型関数 くし型関数（くしがたかんすう、）は、デルタ関数を一定の間隔で並べた超関数。英語か...
1,mathematics,"フィッティングの補題 数学において、の補題 (FITTING LEMMA) は、""M"" が直..."
2,philosophy,理性 理性（りせい、→→→）とは、人間に本来的に備わっているとされる知的能力の一つである。言...
3,mathematics,位相線型環 数学の函数解析学における位相線型環（いそうせんけいかん、; 位相多元環、位相代数...
4,mathematics,調和解析 数学の一分野としての調和解析（ちょうわかいせき、）は、関数や信号を基本波の重ね合わ...


In [6]:
# 訓練データ全体に対してわかち書きを行ったものを新しく列に追加していく
mecab = MeCab.Tagger("-O wakati")

# わかち書きした文章をいれる
text_tokenized = []
for text in df["text"]:
    #parse():わかち書きを行うメソッド
    text_tokenized.append(mecab.parse(text))

# 新しく列を追加する    
df["text_tokenized"] = text_tokenized

In [7]:
df.head()

,category,text,text_tokenized
0,mathematics,くし型関数 くし型関数（くしがたかんすう、）は、デルタ関数を一定の間隔で並べた超関数。英語か...,くし 型 関数 くし 型 関数 （ くし が たかん すう 、 ） は 、 デルタ 関数 を...
1,mathematics,"フィッティングの補題 数学において、の補題 (FITTING LEMMA) は、""M"" が直...",フィッティング の 補題 数学 において 、 の 補題 ( FITTING LEMMA ) ...
2,philosophy,理性 理性（りせい、→→→）とは、人間に本来的に備わっているとされる知的能力の一つである。言...,理性 理性 （ り せい 、 → → → ） と は 、 人間 に 本来 的 に 備わっ て...
3,mathematics,位相線型環 数学の函数解析学における位相線型環（いそうせんけいかん、; 位相多元環、位相代数...,位相 線型 環 数学 の 函数 解析 学 における 位相 線型 環 （ い そう せん けい...
4,mathematics,調和解析 数学の一分野としての調和解析（ちょうわかいせき、）は、関数や信号を基本波の重ね合わ...,調和 解析 数学 の 一 分野 として の 調和 解析 （ ちょう わかい せき 、 ） は...


In [8]:
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC
from sklearn import grid_search

/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
# tf-idf
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
x_tfidf = vectorizer.fit_transform(df["text_tokenized"])

In [10]:
# idf値の確認
vectorizer.idf_

array([2.64089662, 7.2156076 , 5.7115302 , ..., 7.2156076 , 6.81014249,
       6.81014249])

In [35]:
vectorizer.vocabulary_.items()

dict_items([('くし', 6144), ('型', 18802), ('関数', 30013), ('が', 6073), ('たかん', 6613), ('すう', 6449), ('は', 7144), ('デルタ', 11461), ('を', 7843), ('一定', 14905), ('の', 7116), ('間隔', 29996), ('で', 6838), ('並べ', 15334), ('た', 6594), ('超', 28848), ('英語', 27409), ('から', 6053), ('コム', 9836), ('とも', 6910), ('概', 23519), ('形', 20825), ('キリル', 9357), ('文字', 22466), ('ш', 5583), ('に', 7056), ('たとえ', 6650), ('て', 6823), ('シャー', 10264), ('しゃ', 6402), ('ー', 14873), ('かんす', 6067), ('う', 5825), ('と', 6857), ('も', 7506), ('呼ば', 18361), ('れる', 7803), ('また', 7397), ('わかり', 7819), ('やすく', 7590), ('周期', 18345), ('的', 25447), ('連続', 29323), ('積', 26104), ('取る', 17934), ('こと', 6253), ('により', 7084), ('離散', 30265), ('化', 17502), ('サンプリング', 10106), ('し', 6355), ('数値', 22415), ('列', 17088), ('得る', 20972), ('できる', 6844), ('この', 6262), ('ため', 6668), ('理想', 25055), ('な', 6973), ('サンプラー', 10105), ('モデル', 13912), ('として', 6884), ('扱わ', 21703), ('成り立つ', 21501), ('フィッティング', 12522), ('補題', 27791), ('数学', 22421), ('において', 7060)

In [11]:
# idf値の昇順で単語を確認する（インデックスに文字列、idfカラムにidf値）
# sorted()のkey=lambdaは何を基準にソートするのかを決める
pd.Series(vectorizer.idf_, index= [x[0] for x in sorted(vectorizer.vocabulary_.items(),key=lambda x:x[1])]).to_frame("idf").sort_values("idf",ascending=False)

,idf
リゾーマタ,7.215608
下がら,7.215608
下人,7.215608
下中,7.215608
下ろす,7.215608
下の句,7.215608
下せる,7.215608
下さい,7.215608
下げ,7.215608
雅男,7.215608


In [12]:
# テストデータの読み込み
df_test = pd.read_csv("wikipedia-test.txt",sep="\t")
df_test.head()

,category,text
0,philosophy,細見和之 細見和之（ほそみ かずゆき、1962年2月27日）は、日本の詩人、京都大学教授、大...
1,mathematics,グライバッハ標準形 計算機科学において、文脈自由言語の全ての生成規則が次のように書けるとき、...
2,mathematics,"淡中圏 淡中圏（たんなかけん、TANNAKIAN CATEGORY）とは与えられた体""K""に..."
3,mathematics,根岸世雄 根岸 世雄（ねぎし ときお、1929年 - 2005年1月26日）は、日本の数学者...
4,mathematics,CEF CEF


In [13]:
mecab = MeCab.Tagger("-O wakati")

text_tokenized = []

for text in df_test["text"]:
    text_tokenized.append(mecab.parse(text))

df_test["text_tokenized"] = text_tokenized

In [14]:
df_test.head()

,category,text,text_tokenized
0,philosophy,細見和之 細見和之（ほそみ かずゆき、1962年2月27日）は、日本の詩人、京都大学教授、大...,細見 和之 細見 和之 （ ほそ み か ず ゆき 、 1962 年 2 月 27 日 ...
1,mathematics,グライバッハ標準形 計算機科学において、文脈自由言語の全ての生成規則が次のように書けるとき、...,グライバッハ 標準 形 計算 機 科学 において 、 文脈 自由 言語 の 全て の 生成 ...
2,mathematics,"淡中圏 淡中圏（たんなかけん、TANNAKIAN CATEGORY）とは与えられた体""K""に...",淡 中 圏 淡 中 圏 （ たん な かけ ん 、 TANNAKIAN CATEGORY ）...
3,mathematics,根岸世雄 根岸 世雄（ねぎし ときお、1929年 - 2005年1月26日）は、日本の数学者...,根岸 世 雄 根岸 世 雄 （ ねぎ し ときお 、 1929 年 - 2005 年 1 月...
4,mathematics,CEF CEF,CEF CEF \n


In [15]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X= vectorizer.fit_transform(df["text_tokenized"])

In [16]:
# テストデータに対しても同様にtf-idfを求める
# Xと次元数を合わせるために、transformを使用する（つまり、テストデータにある新しい単語の情報は消失する）
X_test = vectorizer.transform(df_test["text_tokenized"])

In [17]:
# データサイズの確認
# 文章×単語のtf-idf値の行列
print(X.toarray().shape)
print(X_test.toarray().shape)

(1000, 30875)
(100, 30875)


In [ ]:
# 各文章毎の単語のtf-idf値を入力として、テストデータの文章のカテゴリーを予測する
# ロジスティック回帰
# SVM
# ランダムフォレスト
# ニューラルネットワーク

In [21]:
def show_evaluation_metrics(y_true,y_pred):
    print("Accuracy")
    print(accuracy_score(y_true,y_pred))
    print()
    
    print("Report")
    print(classification_report(y_true,y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true,y_pred))

In [ ]:
# ロジスティック回帰
# 最適化を行うパラメータ
# C : 正則化項

In [19]:
# グリッドサーチを用いたパラメータの最適化
params=np.arange(1,200,1)
clf_lr = grid_search.GridSearchCV(LogisticRegression(random_state=0),param_grid={'C':params})
clf_lr.fit(X,df["category"])
print("best_param:\n{}".format(clf_lr.best_params_))
print("best_estimator:\n{}".format(clf_lr.best_estimator_))
print("best_score:\n{}".format(clf_lr.best_score_))

best_param:
{'C': 99}
best_estimator:
LogisticRegression(C=99, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
best_score:
0.933


In [22]:
clf_lr = LogisticRegression(n_jobs=-1,C = 99)
clf_lr.fit(X,df["category"])
y_test_pred = clf_lr.predict(X_test)
show_evaluation_metrics(df_test["category"],y_test_pred)

Accuracy
0.93

Report
             precision    recall  f1-score   support

mathematics       0.94      0.96      0.95        67
 philosophy       0.91      0.88      0.89        33

avg / total       0.93      0.93      0.93       100

Confusion matrix:
[[64  3]
 [ 4 29]]


/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [85]:
# SVM（線形カーネル）
# parameter: 
# penalty	罰則項。L1正則化・L2正則化(デフォルト)を選択可
# loss	評価関数。ヒンジ損失か二乗ヒンジ(デフォルト)
# dual	双対問題を解くか否か(デフォルトはtrue)
# tol	アルゴリズムの終了条件(default=1e-4)
# C	ソフトマージンの厳しさを表すパラメータ

In [87]:
# 最適化を行うパラメータ
# 1 : penalty	罰則項。L1正則化・L2正則化(デフォルト)を選択可
# 2 : loss	評価関数。ヒンジ損失か二乗ヒンジ(デフォルト)
# 3 : C	ソフトマージンの厳しさを表すパラメータ

In [23]:
# グリッドサーチによるパラメータの最適化
Standard = LinearSVC(penalty='l2',loss='hinge',random_state=0)
Penalty_L1 = LinearSVC(penalty='l1',loss='squared_hinge',dual=False,random_state=0)
Penalty_L2 = LinearSVC(penalty='l2',loss='squared_hinge',random_state=0)

model_set = [Standard,Penalty_L1,Penalty_L2]

params =[0.01, 0.1, 1.0,100] 

for model in model_set:
    clf_svc = grid_search.GridSearchCV(model,param_grid={'C':params})
    clf_svc.fit(X,df["category"])
    
    print("best_param:\n{}".format(clf_svc.best_params_))
    print("best_estimator:\n{}".format(clf_svc.best_estimator_))
    print("best_score:\n{}".format(clf_svc.best_score_))

best_param:
{'C': 1.0}
best_estimator:
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=0, tol=0.0001, verbose=0)
best_score:
0.942
best_param:
{'C': 1.0}
best_estimator:
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=0, tol=0.0001,
     verbose=0)
best_score:
0.919
best_param:
{'C': 100}
best_estimator:
LinearSVC(C=100, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)
best_score:
0.937


In [24]:
# グリッドサーチによる最適なパラメータでのテストデータに対する正解率
clf_svc_best = LinearSVC(penalty='l2',C=1.0,loss='hinge',random_state=0)
clf_svc_best.fit(X,df["category"])
y_test_pred = clf_svc_best.predict(X_test)
show_evaluation_metrics(df_test["category"],y_test_pred)

Accuracy
0.93

Report
             precision    recall  f1-score   support

mathematics       0.93      0.97      0.95        67
 philosophy       0.93      0.85      0.89        33

avg / total       0.93      0.93      0.93       100

Confusion matrix:
[[65  2]
 [ 5 28]]


In [ ]:
# ランダムフォレスト
# 最適化を行うparameter: 
# n_estimators:木をいくつ生成するか。デフォルトでは10。
# max_depth:木の深さの設定。デフォルトはなし
# max_features:分岐に用いる説明変数の数を設定。デフォルトは自動。
# min_sample_split:分割する際の最小のサンプル数を設定。デフォルトは2。

In [25]:
params = {
    'n_estimators' :[100,200,300],
    'max_depth':[1,5,10,None]
#     'max_features':[1,5,10]
#     'min_sample_split':[1,2,4]
}

clf_rf = grid_search.GridSearchCV(RandomForestClassifier(random_state=0,n_jobs=-1),params)
clf_rf.fit(X,df["category"])
print("best_param:\n{}".format(clf_rf.best_params_))
print("best_score:\n{}".format(clf_rf.best_score_))

best_param:
{'max_depth': None, 'n_estimators': 200}
best_score:
0.909


In [26]:
clf_rf = RandomForestClassifier(n_estimators=200,max_depth=None,n_jobs=-1,random_state=0)
clf_rf.fit(X,df["category"])
y_test_pred = clf_rf.predict(X_test)
show_evaluation_metrics(df_test["category"],y_test_pred)

Accuracy
0.89

Report
             precision    recall  f1-score   support

mathematics       0.88      0.97      0.92        67
 philosophy       0.92      0.73      0.81        33

avg / total       0.89      0.89      0.89       100

Confusion matrix:
[[65  2]
 [ 9 24]]


In [ ]:
# ニューラルネットワーク

In [27]:
from chainer import Chain,Variable
import chainer.links as L
import chainer.functions as F
from chainer.datasets import TupleDataset
from chainer.iterators import SerialIterator
from chainer.training import StandardUpdater,Trainer
from chainer.training.extensions import PrintReport,LogReport,Evaluator
from sklearn.preprocessing import StandardScaler
from chainer.datasets import TupleDataset
from chainer.optimizers import Adam
from sklearn.metrics import classification_report

/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [28]:
category_type = df["category"].unique()

In [29]:
category_to_id = dict(zip(category_type,np.arange(category_type.shape[0])))

In [30]:
Y = df["category"].map(lambda x:category_to_id[x])
Y_test = df_test["category"].map(lambda x:category_to_id[x])

In [31]:
X = X.astype(np.float32)
X_test = X_test.astype(np.float32)
Y = Y.astype(np.int32)
Y_test = Y_test.astype(np.int32)

In [32]:
train  = TupleDataset(X.toarray(),Y)
test = TupleDataset(X_test.toarray(),Y_test)

In [33]:
len(train[0][0])

30875

In [34]:
class Model(Chain):
    def __init__(self):
        super(Model,self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(30875,200)
            self.l2 = L.Linear(200,100)
            self.l3 = L.Linear(100,10)
            self.l4 = L.Linear(10,2)
            
    def __call__(self,x):
        h = F.dropout(F.relu(self.l1(x)),ratio=0.5)
        h = F.dropout(F.relu(self.l2(h)),ratio=0.5)
        h = F.dropout(F.relu(self.l3(h)),ratio=0.5)
        return self.l4(h)

In [35]:
train_iterator = SerialIterator(train,batch_size=100,repeat=True)
test_iterator = SerialIterator(test,batch_size = 100,shuffle=False,repeat=False)

In [36]:
model = Model()
classifier = L.Classifier(model)
optimizer = Adam()
optimizer.setup(classifier)
updater = StandardUpdater(train_iterator,optimizer,loss_func=classifier)
trainer = Trainer(updater,(1001,'iteration'))
trainer.extend(Evaluator(test_iterator,classifier),trigger=(100,'iteration'),name='test')
trainer.extend(PrintReport(['iteration','main/loss','test/main/loss', 'main/accuracy', 'test/main/accuracy']))
logreport = LogReport(trigger=(100,'iteration'))
trainer.extend(logreport)
trainer.run()

iteration   main/loss   test/main/loss  main/accuracy  test/main/accuracy
100         0.314665    0.219673        0.8486         0.92                
200         0.0477379   0.36546         0.99           0.92                
300         0.0431816   0.369336        0.9917         0.91                
400         0.0390786   0.403125        0.9926         0.91                
500         0.0341438   0.43661         0.993          0.91                
600         0.0326273   0.438809        0.9927         0.91                
700         0.030726    0.5341          0.9946         0.92                
800         0.0273112   0.550116        0.9929         0.92                
900         0.0287428   0.585356        0.9927         0.91                
1000        0.025566    0.58413         0.9932         0.92                


In [37]:
y = model(X_test.toarray())

In [38]:
pred = np.argmax(y.data,axis=1)

In [42]:
show_evaluation_metrics(Y_test,pred)

Accuracy
0.89

Report
             precision    recall  f1-score   support

          0       0.91      0.93      0.92        67
          1       0.84      0.82      0.83        33

avg / total       0.89      0.89      0.89       100

Confusion matrix:
[[62  5]
 [ 6 27]]


In [187]:
# モデルの保存
# ロジスティック回帰モデルの保存
joblib.dump(clf_lr,"wikipedia_category_logistic.pkl.gz")
joblib.dump(clf_svc_best,"wikipedia_category_svc.pkl.gz")
joblib.dump(clf_rf,"wikipedia_category_rf.pkl.gz")
joblib.dump(model,"wikipedia_category_NN.pkl.gz")

['wikipedia_category_NN.pkl.gz']

In [193]:
# モデルの読み込み
clf_lr_restored = joblib.load("wikipedia_category_logistic.pkl.gz")
clf_svc_restored = joblib.load("wikipedia_category_svc.pkl.gz")
clf_rf_restored = joblib.load("wikipedia_category_rf.pkl.gz")
clf_NN_restored = joblib.load("wikipedia_category_NN.pkl.gz")
print("*****clf_lr******")
print(clf_lr_restored)
print("*****clf_svc******")
print(clf_svc_restored)
print("*****clf_rf******")
print(clf_rf_restored)
print("*****clf_NN******")
print(clf_NN_restored)

*****clf_lr******
LogisticRegression(C=99, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
*****clf_svc******
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=0, tol=0.0001, verbose=0)
*****clf_rf******
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
*****clf_NN******
